# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,5.20,58,100,1.54,US,1699500360
1,1,pakisaji,-8.0665,112.5981,32.11,47,99,2.35,ID,1699500498
2,2,port alfred,-33.5906,26.8910,12.32,94,29,4.62,ZA,1699500498
3,3,bredasdorp,-34.5322,20.0403,11.36,81,0,1.05,ZA,1699500372
4,4,crucecita,-28.9500,-58.9833,22.69,91,2,5.01,AR,1699500498


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", size="Humidity", alpha=0.5, color="City", geo=True, tiles="OSM", height=900, width=1200)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[(city_data_df['Cloudiness'] == 0)  & (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27)  & (city_data_df['Wind Speed'] < 4.5) ,:]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,saint-pierre,-21.3393,55.4781,25.66,69,0,1.54,RE,1699500502
134,134,remire-montjoly,4.9167,-52.2667,25.02,94,0,0.00,GF,1699500555
194,194,belmonte,-15.8631,-38.8828,22.50,78,0,2.46,BR,1699500573
272,272,black point,-34.6187,137.8972,21.04,56,0,3.75,AU,1699500595
279,279,arish,31.1316,33.7984,21.72,80,0,1.80,EG,1699500597
313,313,les avirons,-21.2418,55.3394,24.03,69,0,1.54,RE,1699500609
384,384,san ignacio,17.1561,-89.0714,23.08,94,0,0.00,BZ,1699500639


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotels_df = ideal_cities_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotels_df.insert(column='Hotel Name',loc=5,value='')

# Display sample data
hotels_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,saint-pierre,RE,-21.3393,55.4781,69,
134,remire-montjoly,GF,4.9167,-52.2667,94,
194,belmonte,BR,-15.8631,-38.8828,78,
272,black point,AU,-34.6187,137.8972,56,
279,arish,EG,31.1316,33.7984,80,
313,les avirons,RE,-21.2418,55.3394,69,
384,san ignacio,BZ,17.1561,-89.0714,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000

# set up a parameters dictionary
params = {
    "categories": 'accommodation.hotel',
    "limit": 20,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotels_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotels_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotels_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotels_df.loc[index, 'City']} - nearest hotel: {hotels_df.loc[index, 'Hotel Name']}")

# Display sample data
hotels_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
remire-montjoly - nearest hotel: Complexe Belova
belmonte - nearest hotel: No hotel found
black point - nearest hotel: No hotel found
arish - nearest hotel: سينا
les avirons - nearest hotel: Floralys
san ignacio - nearest hotel: Rainforest Haven Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
18,saint-pierre,RE,-21.3393,55.4781,69,Tropic Hotel
134,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
194,belmonte,BR,-15.8631,-38.8828,78,No hotel found
272,black point,AU,-34.6187,137.8972,56,No hotel found
279,arish,EG,31.1316,33.7984,80,سينا
313,les avirons,RE,-21.2418,55.3394,69,Floralys
384,san ignacio,BZ,17.1561,-89.0714,94,Rainforest Haven Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotels_df.hvplot.points("Lng", "Lat", size="Humidity", alpha=0.5, color="City", geo=True, tiles="OSM", height=900, width=1200, hover_cols=['Hotel Name', 'Country'])

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)